<a href="https://colab.research.google.com/github/yulucy19/Thinkful-Project-2019/blob/master/Text_generation_Twitter_US_Airline_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In the following assignments, you're required to use Twitter US Airline Sentiment data. You should access the dataset from the Thinkful database using the following credentials:**

In [52]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
import spacy
import re
import markovify

import warnings
warnings.filterwarnings(action="ignore")

!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [53]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

tweets_df = pd.read_sql_query('select * from twitter',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

pd.options.display.max_colwidth = None
tweets_df.text.head(3)
tweets_df.head(5)

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials to the experience... tacky.,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing about it,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


### 1. First, make some data preprocessing to clean up the data if you feel necessary. You can use your solution to the assignment of the data preprocessing checkpoint of this module.

In [0]:
nlp = spacy.load('en')
#nlp = spacy.load('en', disable=['parser', 'ner'])

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while
twitter_doc = nlp(" ".join(tweets_df.text))

In [55]:
# let's explore the objects we've built.
print("The twitter_doc object is a {} object.".format(type(twitter_doc)))
print("It is {} tokens long".format(len(twitter_doc)))
print("The first three tokens are '{}'".format(twitter_doc[:3]))
print("The type of each token is {}".format(type(twitter_doc[0])))

The twitter_doc object is a <class 'spacy.tokens.doc.Doc'> object.
It is 307328 tokens long
The first three tokens are '@VirginAmerica What @dhepburn'
The type of each token is <class 'spacy.tokens.token.Token'>


### 2. train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences. Are the generated sentences exhibit the same negative sentiment?

In [0]:
# Load the negative sentiment tweets.
twitter_negative_doc = nlp(" ".join(tweets_df[tweets_df["airline_sentiment"]=="negative"].text))

tweet_negative_sents = " ".join([sent.text for sent in twitter_negative_doc.sents if len(sent.text) > 1])

In [57]:
# Generator the negative sentiment.
neg_sentiment_generator = markovify.Text(tweet_negative_sents, state_size = 3)

# Print 20 randomly-generated negative sentiments.
for i in range(20):
    print(neg_sentiment_generator.make_sentence(tries=100))
    
# Print 5 randomly-generated negative sentiments of no more than 100 characters
for i in range(5):
    print(neg_sentiment_generator.make_short_sentence(100))

Will not be flying you again I have been rebooked for tomorrow.
Not only was 5418 Late Flight, but we've been boarded and waiting for my bag, they don't even know why it's behind schedule.
#delayed #again #needtocatchmynextflight #alreadyrebookedonce @United why is it so hard to SPEND money through US Air.
“@united: @qlyss8 We don't want to chance it.
Get a plumber @united I will be sharing this dreadful experience.
Cancelled Flightled flight #poor customerservice @USAirways I'm glad you're sorry that I'm homeless for the night.
I only hope and pray my bag is still in Durango and hasn't departed.
My claim number is expired because you WON'T find my bag @united I had to sleep at the airport right now.
I am yet to be in la tonight!
@AmericanAir I was on hold for more than an hour.
Now I have a girlfriend -_- @JetBlue received a voucher but if you want particulars @usairways @AmericanAir LAX connect from term 6 to term 4.
@AmericanAir Cancelled Flightled flight with a scolding for using a

The results aer not bad, it generated the negative sentiments. 

### 3. Do the previous task this time bu using only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.

In [0]:
# Load the positive sentiment tweets.
twitter_positive_doc = nlp(" ".join(tweets_df[tweets_df["airline_sentiment"]=="positive"].text))

tweet_positive_sents = " ".join([sent.text for sent in twitter_positive_doc.sents if len(sent.text) > 1])

In [59]:
# Generator the positive sentiments.
positive_sentiment_generator = markovify.Text(tweet_positive_sents, state_size = 3)

# Print 20 randomly-generated sentiments.
for i in range(20):
    print(positive_sentiment_generator.make_sentence(tries=100))
    
# Print 5 randomly-generated positive sentiments of no more than 100 characters
for i in range(5):
    print(positive_sentiment_generator.make_short_sentence(100))

@united Thanks for the reply and the follow.
I finally made it to my destination @USAirways thanks for the follow up @JetBlue thanks for your help!
@USAirways your team member at DCA- Tamara R. is her name was Samantha and she won over everyone on the flight @JetBlue I will!
@SouthwestAir thanks for the info...
This is the best part of flying!
@USAirways your team member at DCA- Tamara R. is her name was Samantha and she won over everyone on the flight leaving from Boston to Seattle right now.
#moneyelsewhere @united She met me from customer service at its finest @USAirways Fabulous - thank you so much that helps a ton.
@AmericanAir Well, you guys are great can't wait to hear back from you regarding the internship opportunity!
@VirginAmerica you have the absolute best customer service!
She was able to change my flight 5 times for free!
Was able to get rescheduled.
❤️ no baggage fees.. but need to have more flight time options @SouthwestAir thank you!!! @SouthwestAir @love_dragonss oh m

It looks a good job for positive sentiments.

### 4.This time train your Markov chain model using all the tweets and generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [0]:
# Load the sentiment data
twitter_doc = nlp(" ".join(tweets_df.text))
tweet_sents = " ".join(sent.text for sent in twitter_doc if len(sent.text)>1)

In [63]:
# Generator sentiments.
sentiment_generator = markovify.Text(tweet_sents, state_size = 3)

# # Print 20 randomly-generated sentiments.
for i in range(30):
  print(sentiment_generator.make_sentence(tries=100))

# Print 5 randomly-generated sentiments of no more than 100 characters
for i in range(6):
  print(sentiment_generator.make_short_sentence(100, tries = 100))

@USAirways do nt need to check status of my flight because crew timed out right before my 4hr flight ....
Part of @USAirways lol thanks to you we had to entertain ourselves with witty repartee ...
@united ... she could not access seat map so she said she would get me preferred choice seat for today ...
@united ... she could not access seat map so she said she would get me preferred choice seat for today ...
@AmericanAir Yes but should be able to view my reservation online ....
Part of @USAirways lol thanks to you we had to entertain ourselves with witty repartee ...
@USAirways do nt need to check status of my flight because crew timed out right before my 4hr flight ....
@SouthwestAir my first flight with you today but it was Cancelled Flightled paid for first class ticket but new seat is 38E.
Ca n't believe you would n't take full fare for first class ticket but new seat is 38E.
It takes 13.5 hours to fly from Vegas to Omaha on SW ...
@united how can your app show arriving aircraft is 

Intersting, now you can see both negative and positive sentiments.